# Treasury Yield Tracker

## Introduction

The project automates the process of downloading Treasury rate data and displaying the past interest rate since year-end 2022.  
It consists of sections for installing necessary libraries such as pandas, matplotlib, and yfinance, downloading the Treasury rate data using yfinance, preprocessing the data for visualization, and creating a line plot to visualize the data.

Source: Yahoo Finance
- ^IRX: 13 week T-Bill
- ^FVX: 5 year Treasury
- ^TNX: 10 year Treasury
- ^TYX: 30 year Treasury

## Requirements and Import
- pandas
- numpy
- matplotlib
- yfinance
- altair

In [1]:
import yfinance as yf
import pandas as pd
#import matplotlib.pyplot as plt
import altair as alt
import numpy as np

## Download Treasury rate data

In [2]:
# Download Treasury rate data from Yahoo Finance
#^IRX: 1-Year Eris SOFR Swap Futures,M (YIAH24.CBT)
#^FVX five year Treasury
#^TNX 10 year Treasury
#^TYX 30 year Treasury

treasury_rate_data = yf.download("^IRX ^FVX ^TNX ^TYX", start="2022-12-29", end=None)

[*********************100%***********************]  4 of 4 completed


In [4]:
print(treasury_rate_data.info())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 498 entries, 2022-12-29 to 2024-12-20
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   (Close, ^FVX)   498 non-null    float64
 1   (Close, ^IRX)   498 non-null    float64
 2   (Close, ^TNX)   498 non-null    float64
 3   (Close, ^TYX)   498 non-null    float64
 4   (High, ^FVX)    498 non-null    float64
 5   (High, ^IRX)    498 non-null    float64
 6   (High, ^TNX)    498 non-null    float64
 7   (High, ^TYX)    498 non-null    float64
 8   (Low, ^FVX)     498 non-null    float64
 9   (Low, ^IRX)     498 non-null    float64
 10  (Low, ^TNX)     498 non-null    float64
 11  (Low, ^TYX)     498 non-null    float64
 12  (Open, ^FVX)    498 non-null    float64
 13  (Open, ^IRX)    498 non-null    float64
 14  (Open, ^TNX)    498 non-null    float64
 15  (Open, ^TYX)    498 non-null    float64
 16  (Volume, ^FVX)  498 non-null    int64  
 17  (Volume, ^IRX)  

In [5]:
# The downloaded Treasury rate data
df = treasury_rate_data["Close"]
#df.info()
#print(df)

In [6]:
# Convert the data
# Add Close Date information
df = df.copy()
df['Close Date'] = df.index.strftime('%Y-%m-%d')

# Rename column names
new_column_names = {'^IRX': '13-week T-Bill',
                    '^FVX': '5-yr Treasury',
                    '^TNX': '10-yr Treasury',
                    '^TYX': '30-yr Treasury'}

df = df.rename(columns = new_column_names)

# Output columns
output_list = ['13-week T-Bill', '5-yr Treasury', '10-yr Treasury', '30-yr Treasury']

#df.info()
#print(df)

## Visualize the data

In [7]:
# Melt the DataFrame to convert it to long format
melted_df = df.melt(id_vars='Close Date', var_name='Ticker', value_name='Yield')

# Define range:
# Determine the minimum and maximum values of 'Yield'
min_y_value = melted_df['Yield'].min()
max_y_value = melted_df['Yield'].max()
# Calculate the starting and end points for the Y-axis range
y_start = np.floor(min_y_value*2)/2
y_end = np.ceil(max_y_value*2)/2


# Create a common chart object
chart = alt.Chart(melted_df).mark_line().encode(
    x='Close Date:T',
    y=alt.Y('Yield:Q', scale=alt.Scale(domain=[y_start, y_end])), 
    #y='Yield:Q',
    color=alt.Color('Ticker:N', sort=output_list),
    tooltip=['Close Date:T', 'Ticker:N', 'Yield:Q']
).properties(
    width=600,
    height=400,
    title='Treasury Yields Over Time'
)

# Draw the line
chart

alt.Chart(...)

In [8]:
# Plotting the data
'''
plt.figure(figsize=(12, 6))
plt.plot(df.index, df['13-week T-Bill'], label='13-week T-Bill', color='darkblue')
plt.plot(df.index, df['5-yr Treasury'], label='5-yr Treasury', color='orange')
plt.plot(df.index, df['10-yr Treasury'], label='10-yr Treasury', color='tomato')
plt.plot(df.index, df['30-yr Treasury'], label='30-yr Treasury', color='darkred')
plt.xlabel('Date')
plt.ylabel('Yield')
plt.legend()
plt.grid(True)
plt.show()
'''

"\nplt.figure(figsize=(12, 6))\nplt.plot(df.index, df['13-week T-Bill'], label='13-week T-Bill', color='darkblue')\nplt.plot(df.index, df['5-yr Treasury'], label='5-yr Treasury', color='orange')\nplt.plot(df.index, df['10-yr Treasury'], label='10-yr Treasury', color='tomato')\nplt.plot(df.index, df['30-yr Treasury'], label='30-yr Treasury', color='darkred')\nplt.xlabel('Date')\nplt.ylabel('Yield')\nplt.legend()\nplt.grid(True)\nplt.show()\n"

In [9]:
# Filter for month ends
month_end_data = df.resample('ME').last()
month_end_data.set_index('Close Date', inplace=True)

In [10]:
# Show the filtered DataFrame
print(month_end_data[output_list])

Ticker      13-week T-Bill  5-yr Treasury  10-yr Treasury  30-yr Treasury
Close Date                                                               
2022-12-30           4.260          4.000           3.879           3.975
2023-01-31           4.570          3.638           3.529           3.661
2023-02-28           4.710          4.168           3.916           3.931
2023-03-31           4.585          3.611           3.494           3.689
2023-04-28           4.923          3.536           3.452           3.678
2023-05-31           5.248          3.743           3.637           3.856
2023-06-30           5.153          4.133           3.819           3.855
2023-07-31           5.248          4.179           3.959           4.019
2023-08-31           5.298          4.244           4.093           4.204
2023-09-29           5.300          4.606           4.573           4.711
2023-10-31           5.320          4.816           4.875           5.022
2023-11-30           5.238          4.